In [53]:
from __future__ import unicode_literals, print_function, division

import random
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [56]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [57]:
# clean data as pol.txt contains CC-BY copyright in third column
import pandas

df = pandas.read_csv('pol.txt', sep='\t', header=None)
df = df[[0, 1]]
df.to_csv('eng-pol.txt', sep='\t', header=False, index=False)

In [58]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


In [59]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re",
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [60]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData("eng", "pol", True)
print(random.choice(pairs))

Reading lines...
Read 49943 sentence pairs
Trimmed to 4021 sentence pairs
Counting words...
Counted words:
pol 3435
eng 2212
['martwie sie troche o tom a', 'i m little worried about tom']


In [61]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [62]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=1)
        return decoder_outputs, decoder_hidden, None

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden


In [63]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input, decoder_hidden, encoder_outputs)
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights


In [64]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return input_tensor, target_tensor

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'pol', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device), torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [65]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        encoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [66]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [67]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0

            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [68]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [69]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoder_ids = topi.squeeze()

        decoder_words = []
        for idx in decoder_ids:
            if idx.item() == EOS_token:
                decoder_words.append('<EOS>')
                break
            decoder_words.append(output_lang.index2word[idx.item()])
    return decoder_words, decoder_attn

In [70]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [71]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 49943 sentence pairs
Trimmed to 4021 sentence pairs
Counting words...
Counted words:
pol 3435
eng 2212
0m 18s (- 4m 44s) (5 6%) 6.8667
0m 40s (- 4m 42s) (10 12%) 6.8135
1m 3s (- 4m 36s) (15 18%) 6.8009
1m 23s (- 4m 10s) (20 25%) 6.7897
1m 42s (- 3m 44s) (25 31%) 6.7787
2m 1s (- 3m 23s) (30 37%) 6.7705
2m 21s (- 3m 2s) (35 43%) 6.7611
2m 41s (- 2m 41s) (40 50%) 6.7529
3m 3s (- 2m 22s) (45 56%) 6.7458
3m 24s (- 2m 2s) (50 62%) 6.7374
3m 43s (- 1m 41s) (55 68%) 6.7300
4m 1s (- 1m 20s) (60 75%) 6.7124
4m 20s (- 1m 0s) (65 81%) 6.7003
4m 38s (- 0m 39s) (70 87%) 6.6894
4m 56s (- 0m 19s) (75 93%) 6.6787
5m 14s (- 0m 0s) (80 100%) 6.6752


In [72]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> nie jestem zazdrosna o toma
= i m not jealous of tom
< german SOS SOS SOS SOS SOS SOS SOS SOS SOS

> w niedziele mam wolne
= i m free on sunday
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> jestes wyczerpany
= you re exhausted
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> jestesmy teraz bardzo zajeci
= we re very busy right now
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> on jest bardzo wrazliwy na zimno
= he is very sensitive to cold
< from SOS SOS SOS SOS SOS SOS SOS SOS SOS

> dzis wieczorem lece do australii
= i m leaving tonight for australia
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> jest wystarczajaco bogaty zeby kupic ten obraz
= he is rich enough to buy the painting
< from credible SOS SOS SOS SOS SOS SOS SOS SOS

> on gra muzyke
= he is playing music
< some SOS SOS SOS SOS SOS SOS SOS SOS SOS

> nie wszyscy tutaj sie uczymy
= we re not all students here
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> jestes taktowny
= you re tactful
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS



In [73]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input = ', input_sentence)
    print('output = ', ' '.join(output_words))

    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])

In [77]:
evaluateAndShowAttention('jest wystarczajaco bogaty zeby kupic ten obraz')

evaluateAndShowAttention('on gra muzyke')

evaluateAndShowAttention('jestes taktowny')

evaluateAndShowAttention('nie wszyscy tutaj sie uczymy')

input =  jest wystarczajaco bogaty zeby kupic ten obraz
output =  from credible SOS SOS SOS SOS SOS SOS SOS SOS
input =  on gra muzyke
output =  some SOS SOS SOS SOS SOS SOS SOS SOS SOS
input =  jestes taktowny
output =  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS
input =  nie wszyscy tutaj sie uczymy
output =  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS


C:\Users\mike\AppData\Local\Temp\ipykernel_24524\3122597877.py:7: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
C:\Users\mike\AppData\Local\Temp\ipykernel_24524\3122597877.py:8: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + output_words)
C:\Users\mike\AppData\Local\Temp\ipykernel_24524\3122597877.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\mike\AppData\Local\Temp\ipykernel_24524\3122597877.py:7: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
C:\Users\mike\AppData\Local\Temp\ipykernel_24524\3122597877.py

In [78]:
# Save model

torch.save(encoder.state_dict(), 'encoder.dict')
torch.save(decoder.state_dict(), 'decoder.dict')

In [79]:
# Load model

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

encoder.load_state_dict(torch.load('encoder.dict'))
decoder.load_state_dict(torch.load('decoder.dict'))

<All keys matched successfully>

In [80]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> jestes pomocny
= you re helpful
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> krzykna do niej zeby uwaza a
= he shouted to her to be careful
< true SOS SOS SOS SOS SOS SOS SOS SOS SOS

> nie zwraca uwagi na nauczyciela
= he s not paying attention to the teacher
< traffic mean mean mean mean mean mean mean mean mean

> ochryp em od zbyt intensywnego krzyku
= i am hoarse from yelling so much
< i i obese SOS SOS SOS SOS SOS SOS SOS

> jestem pewien ze tom jest na mnie wsciek y
= i m sure tom is mad at me
< i i i i department sure sure sure SOS SOS

> przyszed em kilka minut za wczesnie
= i m a few minutes early
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> my pomozemy tomowi
= we re going to help tom
< nonsmokers SOS SOS SOS SOS SOS SOS SOS SOS SOS

> planujemy zostac tam tydzien
= we re planning to stay for a week
< contributing SOS SOS SOS SOS SOS SOS SOS SOS SOS

> ide do miasta
= i m going to town
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> pojde umyc rece
= i m going to wash my hand